## Lesson 1 Exercise 1: Creating a Table with PostgreSQL

## Install the Dependencies

In [1]:
!pip3 install pandas psycopg2

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


## Clean Up Database to a clean slate

In [2]:
!brew services stop postgresql
!brew services start postgresql
!psql postgres -c "DROP DATABASE langcodes;"
!psql postgres -c "\l"

Stopping `postgresql@14`... (might take a while)
==> Successfully stopped `postgresql@14` (label: homebrew.mxcl.postgresql@14)
==> Successfully started `postgresql@14` (label: homebrew.mxcl.postgresql@14)
DROP DATABASE
                         List of databases
   Name    | Owner | Encoding | Collate | Ctype | Access privileges 
-----------+-------+----------+---------+-------+-------------------
 postgres  | glall | UTF8     | C       | C     | 
 template0 | glall | UTF8     | C       | C     | =c/glall         +
           |       |          |         |       | glall=CTc/glall
 template1 | glall | UTF8     | C       | C     | =c/glall         +
           |       |          |         |       | glall=CTc/glall
(3 rows)



## Import the library

In [3]:
import psycopg2
import pandas as pd

## Defining a method to create database and return a connection and cursor to work upon. This will be executed from later steps

In [4]:
def create_database(dbname, user, password):
    conn = psycopg2.connect(f"host=127.0.0.1 dbname=postgres user={user} password={password}")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    cur.execute(f"DROP DATABASE IF EXISTS {dbname}")
    cur.execute(f"CREATE DATABASE {dbname}")

    conn.close()

    conn = psycopg2.connect(f"host=127.0.0.1 dbname={dbname} user={user} password={password}")
    curr = conn.cursor()

    return curr, conn

## Read from language-codes_csv.csv

In [5]:
language_codes = pd.read_csv("dataset/language-codes-3b2_csv.csv");

## Display first five rows of the language_codes_clean Dataframe created

In [6]:
print(language_codes.head());

  alpha3-b alpha2    English
0      aar     aa       Afar
1      abk     ab  Abkhazian
2      afr     af  Afrikaans
3      aka     ak       Akan
4      alb     sq   Albanian


## Rename Columns for language_codes_clean

In [7]:
language_codes_clean = language_codes.rename(columns={"alpha2":"character-code-2","English":"english-name"});

## Display first five rows of the language_codes_clean Dataframe updated

In [8]:
print(language_codes_clean.head());

  alpha3-b character-code-2 english-name
0      aar               aa         Afar
1      abk               ab    Abkhazian
2      afr               af    Afrikaans
3      aka               ak         Akan
4      alb               sq     Albanian


## Read from language_codes_full csv

In [9]:
language_codes_full = pd.read_csv("dataset/language-codes-full_csv.csv");

## Peek language_codes_full dataframe

In [10]:
print(language_codes_full.head());

  alpha3-b alpha3-t alpha2    English   French
0      aar      NaN     aa       Afar     afar
1      abk      NaN     ab  Abkhazian  abkhaze
2      ace      NaN    NaN   Achinese     aceh
3      ach      NaN    NaN      Acoli    acoli
4      ada      NaN    NaN    Adangme  adangme


## Rename columns of language_codes_full Dataframe

In [11]:
language_codes_full = language_codes_full.rename(columns={"alpha3-b":"character-code-3","alpha2":"character-code-2","English":"english-name","French":"french-name"});

## Clean up alpha3-t column

In [12]:
language_codes_full_clean = language_codes_full[['character-code-3','character-code-2','english-name','french-name']];

## Clean up blank values

In [13]:
language_codes_full_clean = language_codes_full_clean.fillna('');

## Print language_codes_full_clean

In [14]:
print(language_codes_full_clean.head(10));

  character-code-3 character-code-2            english-name  \
0              aar               aa                    Afar   
1              abk               ab               Abkhazian   
2              ace                                 Achinese   
3              ach                                    Acoli   
4              ada                                  Adangme   
5              ady                           Adyghe; Adygei   
6              afa                   Afro-Asiatic languages   
7              afh                                 Afrihili   
8              afr               af               Afrikaans   
9              ain                                     Ainu   

                french-name  
0                      afar  
1                   abkhaze  
2                      aceh  
3                     acoli  
4                   adangme  
5                    adyghé  
6  afro-asiatiques, langues  
7                  afrihili  
8                 afrikaans  
9     

## Read from ietf_language_codes_csv

In [15]:
ietf_language_tags = pd.read_csv("dataset/ietf-language-tags_csv.csv");

## Rename columns of ietf_language_codes

In [16]:
ietf_language_tags = ietf_language_tags.rename(columns={"langType":"lang-type"});

## Clean up unneeded columns

In [17]:
ietf_language_tags_clean = ietf_language_tags[['lang','lang-type','territory','defs','file']];

## Clean up blank values

In [18]:
ietf_language_tags_clean = ietf_language_tags_clean.fillna('');

## Peek ietf_language_tags_clean

In [19]:
print(ietf_language_tags_clean.head(10));

     lang lang-type territory  defs        file
0      af        af              10      af.xml
1   af-NA        af               2   af_NA.xml
2   af-ZA        af        ZA     0   af_ZA.xml
3     agq       agq               6     agq.xml
4  agq-CM       agq        CM     0  agq_CM.xml
5      ak        ak               6      ak.xml
6   ak-GH        ak        GH     0   ak_GH.xml
7      am        am              10      am.xml
8   am-ET        am        ET     0   am_ET.xml
9      ar        ar              11      ar.xml


## Create the database and get a cursor

In [20]:
curr, conn = create_database("langcodes","glall","root");

## Create Function to check custom constraint

In [21]:
try: 
    curr.execute("CREATE OR REPLACE FUNCTION check_lang_type(lang_type varchar) \
                RETURNS boolean AS $$ \
                BEGIN \
                    RETURN EXISTS (SELECT 1 FROM language_codes WHERE character_code_2 = lang_type) \
                        OR EXISTS (SELECT 1 FROM language_codes_extended WHERE character_code_3 = lang_type); \
                END; \
                $$ LANGUAGE plpgsql;")

    curr.execute("CREATE OR REPLACE FUNCTION check_character_code_2(p_character_code_2 varchar) \
                RETURNS boolean AS $$ \
                BEGIN \
                    RETURN p_character_code_2 IS NULL \
                        OR p_character_code_2 = '' \
                        OR EXISTS (SELECT 1 FROM language_codes WHERE character_code_2 = p_character_code_2); \
                END; \
                $$ LANGUAGE plpgsql;")

except psycopg2.Error as e: 
    print("Error: Issue creating custom check constraint")
    print (e)

## Create Tables for Langcodes database

In [22]:
try: 
    curr.execute("CREATE TABLE IF NOT EXISTS language_codes (alpha_3b varchar, character_code_2 varchar PRIMARY KEY, english_name varchar);")

    curr.execute("CREATE TABLE IF NOT EXISTS language_codes_extended (character_code_3 varchar PRIMARY KEY, character_code_2 varchar, english_name varchar, french_name varchar, \
                    CONSTRAINT character_code_2_check CHECK (check_character_code_2(character_code_2)));")
    
    curr.execute("CREATE TABLE IF NOT EXISTS language_codes_ietf (lang varchar PRIMARY KEY, lang_type varchar, territory varchar, defs varchar, file varchar,\
                    CONSTRAINT lang_type_check CHECK (check_lang_type(lang_type)));")

except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

## Insert Dataframes into DB tables

In [23]:
try:
    language_codes_insert = ("""INSERT INTO language_codes(alpha_3b, character_code_2, english_name) VALUES (%s, %s, %s)""")
    
    for i, row in language_codes_clean.iterrows():
        curr.execute(language_codes_insert, list(row))
    
    conn.commit();
    
    
    language_codes_full_insert = ("""INSERT INTO language_codes_extended(character_code_3, character_code_2, english_name, french_name) VALUES (%s, %s, %s, %s)""")
    
    for i, row in language_codes_full_clean.iterrows():
        curr.execute(language_codes_full_insert, list(row))
    
    conn.commit();
    
    
    ietf_language_tags_insert = ("""INSERT INTO language_codes_ietf(lang, lang_type, territory, defs, file) VALUES (%s, %s, %s, %s, %s)""")
    
    for i, row in ietf_language_tags_clean.iterrows():
        curr.execute(ietf_language_tags_insert, list(row))
    
    conn.commit();

except psycopg2.Error as e:
    print("Error while inserting records into tables from dataframes")
    print(e)
    curr.execute("ROLLBACK;");

Error while inserting records into tables from dataframes
new row for relation "language_codes_ietf" violates check constraint "lang_type_check"
DETAIL:  Failing row contains (agq, agq, , 6, agq.xml).



## Relax Constraint on table

In [24]:
try:
    curr.execute("ALTER TABLE language_codes_ietf DROP CONSTRAINT lang_type_check;")

except psycopg2.Error as e: 
    print("Error: Issue deleting constraint from table")
    print (e)

## Retry addition of records into language_codes_ietf table

In [25]:
try:
    
    ietf_language_tags_insert = ("""INSERT INTO language_codes_ietf(lang, lang_type, territory, defs, file) VALUES (%s, %s, %s, %s, %s)""")
    
    for i, row in ietf_language_tags_clean.iterrows():
        curr.execute(ietf_language_tags_insert, list(row))
    
    conn.commit();

except psycopg2.Error as e:
    print("Error while inserting records into tables from dataframes")
    print(e)
    curr.execute("ROLLBACK;");

##  And finally close your cursor and connection.

In [26]:
curr.close()
conn.close()